<a href="https://colab.research.google.com/github/akshat-vashistha/Akshat/blob/main/VLG_openproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!pip install -q  keras-nlp
!pip install -q  keras

In [ ]:
!pip install keras==3.0.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_core as keras
import keras_core.backend as K


import jax
import tensorflow as tf
# from tensorflow import keras
# import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm
import gc

In [ ]:
print("TensorFlow:", tf.__version__)
# print("JAX:", jax.__version__)
print("Keras:", keras.__version__)

TensorFlow: 2.15.0
Keras: 0.1.7


In [ ]:
def get_device():
    "Detect and intializes GPU/TPU automatically"
    try:
        # Connect to TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        # Set TPU strategy
        strategy = tf.distribute.TPUStrategy(tpu)
        print(f'> Running on TPU', tpu.master(), end=' | ')
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        device=CFG.device
    except:
        # If TPU is not available, detect GPUs
        gpus = tf.config.list_logical_devices('GPU')
        ngpu = len(gpus)
         # Check number of GPUs
        if ngpu:
            # Set GPU strategy
            strategy = tf.distribute.MirroredStrategy(gpus) # single-GPU or multi-GPU
            # Print GPU details
            print("> Running on GPU", end=' | ')
            print("Num of GPUs: ", ngpu)
            device='GPU'
        else:
            # If no GPUs are available, use CPU
            print("> Running on CPU")
            strategy = tf.distribute.get_strategy()
            device='CPU'
    return strategy, device

In [ ]:
import pandas as pd
df= pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/test_essays.csv')
df.head()


,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [ ]:
def build_dataset(texts, labels=None, batch_size=32,
                  cache=False, drop_remainder=True,
                  augment=False, repeat=False, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, labels)  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    ds = ds.repeat() if repeat else ds  # Repeat dataset if enabled
    opt = tf.data.Options()  # Create dataset options
    if shuffle:
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

In [ ]:
def build_model():
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        CFG.preset,
        load_weights=False,
        preprocessor=None,
        num_classes=1 # one output per one option, for five options total 5 outputs
    )
    inputs = classifier.input
    logits = classifier(inputs)

    # Compute final output
    outputs = keras.layers.Activation("sigmoid")(logits)
    model = keras.Model(inputs, outputs)
    return model

In [ ]:
# Get the checkpoint directory and name
ckpt_dir = CFG.ckpt_dir
ckpt_name = ckpt_dir.split('/')[3]

# Copy the checkpoints to a new directory in the /kaggle directory
!cp -r {ckpt_dir} /kaggle/{ckpt_name}

# List all the checkpoint paths in the new directory
new_ckpt_dir = f"/kaggle/{ckpt_name}"
ckpt_paths = glob(os.path.join(new_ckpt_dir, '*.keras'))

print("Total CKPT:", len(ckpt_paths))

In [ ]:


# Initialize an array to store predictions for each fold
fold_preds = np.zeros(shape=(len(test_df),), dtype='float32')

# # Build model
# model = build_model()

# Iterate through each checkpoint path
for ckpt_path in tqdm(ckpt_paths):
    # Load the pre-trained model from the checkpoint
    model = keras.models.load_model(
        ckpt_path,
        compile=False,
    )
#     model.load_weights(ckpt_path)

    # Get the test dataset
    test_ds = get_test_dataset(test_df)

    # Generate predictions using the model
    preds = model.predict(
        test_ds,
        batch_size=min(CFG.batch_size * CFG.replicas * 2, len(test_df)),  # Set batch size
        verbose=1
    )

    # Add predictions to fold_preds and average over checkpoints
    fold_preds += preds.squeeze() / len(ckpt_paths)

    # Clean up by deleting the model and collecting garbage
    del model
    gc.collect()

In [ ]:
# Format predictions and true answers
pred_answers = (fold_preds > 0.5).astype(int).squeeze()

# Check 5 Predictions
print("# Predictions\n")
for i in range(3):
    row = test_df.iloc[i]
    text  = row.text
    pred_answer = CFG.label2name[pred_answers[i]]
    print(f"❓ Text {i+1}:\n{text}\n")
    print(f"🤖 Predicted: {pred_answer}\n")
    print("-"*90, "\n")